In [4]:
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# Set the directory path
directory_path = '/Users/dbai033/Desktop/Trade_Project/Final_output'

# Load the data from CSV (replace 'your_data.csv' with your actual file name)
file_path = os.path.join(directory_path, 'BACI_2012_2022_merged_data.csv')
df = pd.read_csv(file_path)

df['i'] = df['i'].astype('category')
df['j'] = df['j'].astype('category')
df['k'] = df['k'].astype('category')

df = df.rename(columns={'t': 'year', 'i': 'exporter', 'j': 'importer', 'k': 'product', 'v': 'value', 'q': 'quantity'})

# Calculate total export value by exporter and year
df['total_exports_by_exporter_by_year'] = df.groupby(['exporter', 'year'])['value'].transform('sum')

# Calculate total export value by product, exporter and year
df['total_exports_by_product_by_exporter_by_year'] = df.groupby(['product', 'exporter', 'year'])['value'].transform('sum')

# Calculate total export value by product and year
df['total_exports_by_product_by_year'] = df.groupby(['product', 'year'])['value'].transform('sum')

# Calculate total export value by year
df['total_exports_by_year'] = df.groupby(['year'])['value'].transform('sum')

# Calculate Revealed Comparative Advantage (RCA)
df['RCA'] = (df['total_exports_by_product_by_exporter_by_year'] / df['total_exports_by_exporter_by_year']) / (df['total_exports_by_product_by_year'] / df['total_exports_by_year'])

# Round to 2 decimal places
df['RCA'] = df['RCA'].round(2)

# Set the directory path for the Parquet file
parquet_file_path = os.path.join(directory_path, 'outputRCAFinal5.parquet')

# Save DataFrame as Parquet file
df.to_parquet(parquet_file_path, engine='pyarrow', compression='GZIP')
